<a href="https://colab.research.google.com/github/nguyenduytung74913/bai-tap-ve-nha-deep-learning/blob/main/b%C3%A0i_2_tu%E1%BA%A7n_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
 
 # Input data 
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("./data/MNIST_data", one_hot=True)
 
 # Define network hyperparameters
learning_rate = 0.0001
training_iters = 400000
batch_size = 256
display_step = 5
 
 # Define network parameters
N_input = 784 # Enter the dimension (img shape: 28*28)
N_classes = 10 #labeled dimensions (0-9 digits)
Dropout = 0.5 # probability of Dropout, possibility of output
 
 # Enter placeholder
x = tf.placeholder(tf.float32, [None, n_input])
y = tf.placeholder(tf.float32, [None, n_classes])
keep_prob = tf.placeholder(tf.float32)   # dropout (keep probability)
 
 
 # Define convolution operations
def conv2d(name, x, W, b, strides=1):
    # Conv2D wrapper, with bias and relu activation
    x = tf.nn.conv2d(x, W, strides=[1, strides, strides, 1], padding='SAME')
    x = tf.nn.bias_add(x, b)
    return tf.nn.relu(x, name=name) # Use relu to activate the function
 
 
 # Define pooling layer operations
def maxpool2d(name, x, k=2):
    # MaxPool2D wrapper
    return tf.nn.max_pool(x, ksize=[1, k, k, 1], strides=[1, k, k, 1],
                          padding='SAME', name=name)
 
 
 # Normalized operation
def norm(name, l_input, lsize=4):
    return tf.nn.lrn(l_input, lsize, bias=1.0, alpha=0.001 / 9.0,
                     beta=0.75, name=name)
 
 # Define all network parameters
weights = {
    'wc1': tf.Variable(tf.random_normal([11, 11, 1, 96])),
    'wc2': tf.Variable(tf.random_normal([5, 5, 96, 256])),
    'wc3': tf.Variable(tf.random_normal([3, 3, 256, 384])),
    'wc4': tf.Variable(tf.random_normal([3, 3, 384, 384])),
    'wc5': tf.Variable(tf.random_normal([3, 3, 384, 256])),
    'wd1': tf.Variable(tf.random_normal([4*4*256, 4096])),
    'wd2': tf.Variable(tf.random_normal([4096, 4096])),
    'out': tf.Variable(tf.random_normal([4096, n_classes]))
}
biases = {
    'bc1': tf.Variable(tf.random_normal([96])),
    'bc2': tf.Variable(tf.random_normal([256])),
    'bc3': tf.Variable(tf.random_normal([384])),
    'bc4': tf.Variable(tf.random_normal([384])),
    'bc5': tf.Variable(tf.random_normal([256])),
    'bd1': tf.Variable(tf.random_normal([4096])),
    'bd2': tf.Variable(tf.random_normal([4096])),
    'out': tf.Variable(tf.random_normal([n_classes]))
}
 
 
 # Define the entire network
def alex_net(x, weights, biases, dropout):
         # Vector to matrix Reshape input picture
    x = tf.reshape(x, shape=[-1, 28, 28, 1])
 
         # 
         # convolution
    conv1 = conv2d('conv1', x, weights['wc1'], biases['bc1'])
         # 
    pool1 = maxpool2d('pool1', conv1, k=2)
         # 
    norm1 = norm('norm1', pool1, lsize=4)
 
         # 
         # convolution
    conv2 = conv2d('conv2', norm1, weights['wc2'], biases['bc2'])
         #Maximum pooling (downsampling)
    pool2 = maxpool2d('pool2', conv2, k=2)
         # 
    norm2 = norm('norm2', pool2, lsize=4)
 
         # third layer convolution
         # convolution
    conv3 = conv2d('conv3', norm2, weights['wc3'], biases['bc3'])
         # 
    norm3 = norm('norm3', conv3, lsize=4)
 
         # 
    conv4 = conv2d('conv4', norm3, weights['wc4'], biases['bc4'])
 
         # fifth layer convolution
    conv5 = conv2d('conv5', conv4, weights['wc5'], biases['bc5'])
         #Maximum pooling (downsampling)
    pool5 = maxpool2d('pool5', conv5, k=2)
         # 
    norm5 = norm('norm5', pool5, lsize=4)
 
         # 1
    fc1 = tf.reshape(norm5, [-1, weights['wd1'].get_shape().as_list()[0]])
    fc1 =tf.add(tf.matmul(fc1, weights['wd1']), biases['bd1'])
    fc1 = tf.nn.relu(fc1)
    # dropout
    fc1 = tf.nn.dropout(fc1, dropout)
 
         # 2
    fc2 = tf.reshape(fc1, [-1, weights['wd2'].get_shape().as_list()[0]])
    fc2 = tf.add(tf.matmul(fc2, weights['wd2']), biases['bd2'])
    fc2 = tf.nn.relu(fc2)
    # dropout
    fc2=tf.nn.dropout(fc2, dropout)
 
         #output layer
    out = tf.add(tf.matmul(fc2, weights['out']), biases['out'])
    return out
 
 #Build model
pred = alex_net(x, weights, biases, keep_prob)
 
 # Define loss function and optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)
 
 # 
correct_pred = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))
 
 # Initialize variables 
init = tf.global_variables_initializer()
 
import os
ckpt_dir = './tmp/ckpt_dir'
if not os.path.exists(ckpt_dir):
    os.makedirs(ckpt_dir)
 # Define an extractor
saver = tf.train.Saver()
 
 # Open a training
with tf.Session() as sess:
    sess.run(init)
    step = 1
         #Start training until training_iters is reached, ie 200000
    while step * batch_size < training_iters:
                 # Get batch data
        batch_x, batch_y = mnist.train.next_batch(batch_size)
        sess.run(optimizer, feed_dict={x: batch_x, y: batch_y, keep_prob: dropout})
        if step % display_step == 0:
                         # Calculate the loss value and accuracy, output
            loss, acc = sess.run([cost, accuracy], feed_dict={x: batch_x, y: batch_y, keep_prob: 1.})
            print("Iter " + str(step*batch_size) + ", Minibatch Loss= " + "{:.6f}".format(loss) + ", Training Accuracy= " + "{:.5f}".format(acc))
        step += 1
    print("Optimization Finished!")
         # Calculate the accuracy of the test set
    print("Testing Accuracy:", sess.run(accuracy, feed_dict={x: mnist.test.images[:256], y: mnist.test.labels[:256], keep_prob: 1.}))
    saver.save(sess, './tmp/ckpt_dir/model.ckpt')

ModuleNotFoundError: ignored